  - 네이버 개발자 api 신청
  - https://developers.naver.com/products/service-api/search/search.md
  - 예제 : https://developers.naver.com/docs/serviceapi/search/blog/blog.md#python

In [1]:
# 네이버 검색 API 예제 - 블로그 검색
import os
import sys
import urllib.request
import json
import html
import pandas as pd

In [2]:
def getNaverSearch(srcText='뉴스',type="news",display=10,start=1):
  client_id = "UZngLtPI8EwAfuzTadat"
  client_secret = "aNw38dR4UQ"
  encText = urllib.parse.quote(srcText)
  url = f"https://openapi.naver.com/v1/search/{type}?query={encText}&display={display}&start={start}" # JSON 결과
  # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()
  if(rescode==200):
      response_body = response.read()
      return json.loads(response_body.decode('utf-8'))
  else:
      print("Error Code:" + rescode)
      return None

In [14]:
def makeCSV(result,*args):
  title_list,originallink_list,link_lilst,description_list = [],[],[],[]
  keys_ = {}
  for data in result['items']:
    for keyname in args:
      if keyname not in keys_:
        keys_[keyname] = []
      keys_[keyname].append(html.unescape( data[keyname]))

  return pd.DataFrame(keys_)

In [16]:
makeCSV(getNaverSearch('홍수'), 'title','description')

,title,description
0,"홍준표 대구시장, 호우 대비 현장 점검","홍 시장은 시와 K-water 직원들을 격려하고, 태풍이나 집중 호우로 <b>홍수<..."
1,주말 호우 대비 주요 댐 안전대책 점검,홍 시장은 댐 관리자들과 현장 회의를 통해 <b>홍수</b> 조절과 물 공급에 관한...
2,"수자원공사, 기후위기 시대 대비 위한 혁신기구 ‘K-water Inno-WAVE 추...","공사는 ‘K-water Inno-WAVE 추진단` 운영을 통해, 기후위기 심화 및 ..."
3,5년 전 ‘수해백서’ 내놓고도 ‘사후약방문’ 실행 안 한 청주,그런데 미호강은 국가 하천이라 금강유역환경청의 하천기본계획에 따라 설계<b>홍수</...
4,"""4대강 보 해체, '反4대강단체'에 휘둘렸다""…환경부 ""16개보 모두 존치""",<b>홍수</b>가 잦아지면서 이명박 정부에서 추진된 4대강 사업을 재평가해야 한다...
5,"대구시 동구청, 천연기념물 도동 측백나무 숲 보존과 활용 토의 진행","동구청에 따르면 ""토의는 기후변화로 인한 가뭄·<b>홍수</b>와 같은 재난 상황에..."
6,"[연합시론] '금강·영산강 보 해체' 감사 결과, 물관리 재검토 불가피",4대강 사업은 이명박 정부 당시 <b>홍수</b> 피해를 예방하고 수자원을 확보하며...
7,"수자원공사 낙동강유역본부, 환경단체 낙동강 보 안전성 우려에 '안전 문제없...",수자원공사는 <b>홍수</b>기가 끝나는 대로 재발을 막기 위한 항구적인 복구 작업...
8,"감사원 “전 정부 환경장관, 4대강위 불공정 구성”",사의재는 “윤석열 정부가 감사원의 맹탕 발표와 <b>홍수</b> 사태를 계기로 국가...
9,"감사원 ""文정부 금강·영산강 보 해체, 환경부 위법 있었다""","◇ 이번이 5번째 감사…""정치 감사 해당 안 돼"" 4대강 사업은 이명박 정부 당시 ..."


In [ ]:
df_lists = []
display = 100
for i in range(10):
  start = i*display+1
  df_lists.append(makeCSV(getNaverSearch('홍수',start=start,display=display)))

리스트에 담긴 dataFrame 병합하기(세로로)

In [ ]:
merge_df = pd.concat(df_lists)
merge_df_du = merge_df.drop_duplicates(subset='title')

In [ ]:
merge_df_du

### 네이버 뉴스검색 말고 응용해서 다른 카테고리의 해당하는 검색 만들기